In [1]:
docs = [
    'I am a student and I live in East China NORMAL University. As a student, I\'m very tired of coding.',
    'I live in Beijing.',
    'I love Japan.'
]

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
t = TfidfVectorizer()

In [3]:
t_matrix = t.fit_transform(docs)

In [5]:
t.get_feature_names_out()

array(['am', 'and', 'as', 'beijing', 'china', 'coding', 'east', 'in',
       'japan', 'live', 'love', 'normal', 'of', 'student', 'tired',
       'university', 'very'], dtype=object)

In [6]:
t.vocabulary_

{'am': 0,
 'student': 13,
 'and': 1,
 'live': 9,
 'in': 7,
 'east': 6,
 'china': 4,
 'normal': 11,
 'university': 15,
 'as': 2,
 'very': 16,
 'tired': 14,
 'of': 12,
 'coding': 5,
 'beijing': 3,
 'love': 10,
 'japan': 8}

In [7]:
t_matrix.toarray()

array([[0.24878394, 0.24878394, 0.24878394, 0.        , 0.24878394,
        0.24878394, 0.24878394, 0.1892066 , 0.        , 0.1892066 ,
        0.        , 0.24878394, 0.24878394, 0.49756788, 0.24878394,
        0.24878394, 0.24878394],
       [0.        , 0.        , 0.        , 0.68091856, 0.        ,
        0.        , 0.        , 0.51785612, 0.        , 0.51785612,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [1]:
import numpy as np
label = np.array([['label', 'pred']])
print(label.shape)

(1, 2)


In [3]:
result = np.array([[1, 2], [3, 4]])
print(result.shape)
np.concatenate(label, result)

(2, 2)


TypeError: only integer scalar arrays can be converted to a scalar index

In [5]:
np.vstack((label, result))

array([['label', 'pred'],
       ['1', '2'],
       ['3', '4']], dtype='<U11')

In [7]:
np.concatenate((label, result), axis=0)

array([['label', 'pred'],
       ['1', '2'],
       ['3', '4']], dtype='<U11')

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import load_files
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
import pandas as pd, numpy as np, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import csv
import datetime
import pickle

In [2]:
# 由于 train_data.txt 与 test.txt 的文件内容组织形式不一样，所以我们需要分别将其转化为统一的 csv 格式
def txt_to_csv_train(filePathSrc, filePathDst):
    list_data = []
    with open(filePathSrc, 'r', encoding='utf-8') as input_file, open(filePathDst, 'w', newline='') as output_file:
        for data in input_file:
            data = eval(data)
            list_data.append(data)
        keys = list_data[0].keys()
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_data)

def txt_to_csv_test(filePathSrc, filePathDst):
    with open(filePathSrc, 'r', encoding='utf-8') as input_file, open(filePathDst, 'w', newline='') as output_file:
        stripped = (line.strip('\n') for line in input_file)
        # 这里只需要用第一个逗号进行分割，因为一个句子中可能有很多个逗号
        lines = (line.split(', ', 1) for line in stripped if line)
        writer = csv.writer(output_file)
        writer.writerows(lines)

In [5]:
txt_to_csv_train('train_data.txt', 'train_data.csv')
txt_to_csv_test('test.txt', 'test.csv')

dataset_train = pd.read_csv('train_data.csv')
X = dataset_train.drop('label', axis=1)
y = dataset_train['label']
# X = X.to_numpy()
X = X.values.tolist()
# print(X)
list1 = []
for index_i in range(len(X)):
    for index_j in range(len(X[index_i])):
        list1.append(X[index_i][index_j])
# print(list1)
tv = TfidfVectorizer(stop_words='english')
X_fit = tv.fit_transform(list1).toarray()
print(X_fit.shape)

# 数据集划分，进行模型的训练以及预测
X_train, X_val, y_train, y_val = train_test_split(X_fit, y, test_size=0.2, random_state=42)
model = SVC(kernel='linear', C=2.0, random_state=42)
print('this model is running')
starttime = datetime.datetime.now()
model.fit(X_train, y_train)
endtime = datetime.datetime.now()
print('this model finishes running, running time:', (endtime - starttime).seconds, 'seconds.')
score = model.score(X_val, y_val)
print("Model score:", score)

(8000, 29697)
this model is running
this model finishes running, running time: 541 seconds.
Model score: 0.958125


In [4]:
grid = GridSearchCV(SVC(kernel='linear'), param_grid={"C":[0.5, 1, 2, 3, 4]}, cv=4)
print('start selecting...')
grid.fit(X_fit, y)
print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

start selecting...
The best parameters are {'C': 2} with a score of 0.89


## Logistic Regression

In [20]:
# 数据集划分，进行模型的训练以及预测
X_train, X_val, y_train, y_val = train_test_split(X_fit, y, test_size=0.2, random_state=42)
model = LogisticRegression(penalty='l2', C=10, random_state=42, max_iter=200)
print('this model is running')
starttime = datetime.datetime.now()
model.fit(X_train, y_train)
endtime = datetime.datetime.now()
print('this model finishes running, running time:', (endtime - starttime).seconds, 'seconds.')
score = model.score(X_val, y_val)
print("Model score:", score)

this model is running
this model finishes running, running time: 41 seconds.
Model score: 0.95125


In [17]:
grid = GridSearchCV(LogisticRegression(max_iter=300), param_grid={"C":[1, 2, 5, 10], "penalty":['l2']}, cv=4)
print('start selecting...')
grid.fit(X_fit, y)
print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

The best parameters are {'C': 1000, 'penalty': 'l2'} with a score of 0.90


D:\pythonDL\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


start selecting...
The best parameters are {'C': 10, 'penalty': 'l2'} with a score of 0.90


In [22]:
# 读取文件 1
with open("results_SVM.txt", "r") as file1:
    lines1 = file1.readlines()

# 读取文件 2
with open("results_LR.txt", "r") as file2:
    lines2 = file2.readlines()

# 转换为集合并获取不同的行
set1 = set(lines1)
set2 = set(lines2)

differences = set1.symmetric_difference(set2)

# 输出不同行的个数
print("使用 SVM 和逻辑斯谛回归训练出来的数据有：", len(differences), "行的不同。")
# 输出不同的行
for line in differences:
    print(line)


使用 SVM 和逻辑斯谛回归训练出来的数据有： 104 行的不同。
459, 6

1267, 5

1440, 9

811, 7

191, 3

1267, 0

968, 8

1030, 3

882, 6

723, 2

328, 3

1658, 8

1981, 0

901, 7

191, 2

1655, 2

901, 3

1010, 3

1010, 2

527, 6

1189, 7

1254, 7

822, 0

968, 4

1455, 8

1858, 4

459, 9

1062, 9

1605, 6

390, 4

1487, 0

522, 8

1898, 6

1605, 9

723, 5

910, 8

1487, 5

1030, 2

889, 5

1025, 8

1189, 2

1542, 5

320, 6

527, 0

328, 4

882, 9

321, 7

622, 9

655, 2

625, 0

52, 7

169, 6

1440, 0

1852, 4

52, 2

1696, 6

655, 7

632, 5

406, 0

1898, 5

889, 1

367, 0

1871, 8

1765, 9

128, 5

1729, 0

1981, 5

1696, 9

822, 5

390, 2

1251, 8

811, 2

599, 5

910, 7

625, 5

667, 0

1729, 5

599, 0

128, 0

169, 0

1542, 0

1858, 2

1025, 2

667, 6

1852, 8

1871, 2

204, 2

321, 2

1251, 3

622, 0

367, 1

320, 0

690, 5

204, 6

1062, 6

1254, 3

1655, 7

1658, 2

522, 3

1765, 6

690, 0

1455, 3

632, 0

406, 6

